<a href="https://colab.research.google.com/github/Selii341/COSC2793A2/blob/main/A2_Sisi_Zhang_S4000130.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1.Data Prepeartaion

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

## 1.1 Loading Data

In [ ]:
#Unzip the File
!unzip -q -o UCI-electricity.zip
!rm UCI-electricity.zip
!ls

A1                           UCI-electricity
A2_Sisi Zhang_S4000130.ipynb __MACOSX


In [ ]:
data = pd.read_csv("UCI-electricity/UCI_data.csv",delimiter=',')

In [ ]:
data.head()

,date,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,...,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2,TARGET_energy
0,2016-04-19 20:30:00,22.200000,39.500000,20.566667,37.656667,22.230000,37.030000,22.318571,36.610000,20.633333,...,33.90,9.70,766.100000,65.5,3.500000,40.000000,3.350000,24.061869,24.061869,60
1,2016-03-05 04:40:00,20.356667,37.126667,17.566667,40.230000,20.890000,37.663333,18.700000,36.260000,18.463333,...,41.09,0.30,740.333333,99.0,1.000000,41.333333,0.100000,4.622052,4.622052,50
2,2016-03-14 12:40:00,20.926667,38.790000,21.100000,35.526667,21.600000,36.290000,21.000000,34.826667,18.100000,...,38.76,4.40,768.466667,72.0,6.000000,22.666667,-0.266667,5.635898,5.635898,80
3,2016-01-22 15:30:00,18.290000,38.900000,17.290000,39.260000,18.390000,39.326667,16.100000,38.790000,16.100000,...,39.20,3.35,760.600000,82.0,5.500000,41.000000,0.500000,49.216445,49.216445,40
4,2016-02-10 00:40:00,22.290000,42.333333,21.600000,40.433333,22.666667,43.363333,19.100000,40.900000,19.290000,...,43.73,3.20,738.900000,88.0,7.333333,56.000000,1.400000,47.617579,47.617579,60


The ID columns is useless here,We will drop it later.

In [ ]:
data.describe()

Describing the dataset. The dataset consists of examples from 193 different countries from different years. It is shown that minimum value for year is 2002 and maximum is 2017; it consists of 15 year period. Also the status column is binary. The population column has huge values in it, which is expected. The values can be showed as millions.However, The SLS dsitribution will be very skewed and some minumum value is zero,we will check them later.

In [ ]:
data.info()

No missing values and all datatype is constructed well.All float are the countinous number,some of int is the Category number i.e. Country,Year and Status.

In [ ]:
data.shape

Now checking shape of dataset which will return the value of number of Rows and Columns.There are 24 Columns and 2071 rows.

## 1.2 Cleaning  Data

In [ ]:
#Status checking
print(set((data['Status']).astype(int)))

In [ ]:
#Missing value
data.isnull().sum()

In [ ]:
#Population Zoom Out
data['Population'] = data['Population']/1000000
data

In [ ]:
#Rename the columns thinness  1-19 years to match the feature description
data.rename(columns={'Thinness1-19years':'Thinness10-19years'}, inplace=True)

In [ ]:
#Checking the data again
data.describe()

In total, AdultMortality, AdultMortality-Male and
AdultMortality-Female are very similar in each value.

In [ ]:
#Looking at the name of columns which can be used later.
data.columns

In [ ]:
#Checking any repeat rows which are meaningless.
data.duplicated().any()

# 2. EDA Data distribution
